In [1]:
# 프로세스
# 운영체제에서 실행되는 프로그램
# 각각 별도의 프로세스
# 하나 이상의 스레드
# 프로세스에 여러개의 스레드가 있다면 마치 동시에 수행하는듯한 느낌을 받을 수 있습니다.
# 멀티 프로세스 : 별도의 메모리 영역을 따로 가지고 있다.
# 특별한 메커니즘으로 통신을 할수있다.
# - 시그널, 메시지 큐, 파이프, IPC 방식
# 프로세서는 각 스레드에서 별도의 레지스터 집합을 부러오거나 저장을 할 수 있습니다.
# 프레세스 간 데이터 공유와 통신용으로 비효율적입니다.
# 멀티 프로세싱 방식에는 subprocess 모듈

# 멀티 스레드 : 단일 프로세스 내의 멀티 슬데느느 동일한 메모리에 접근합니다.
# 스레드는 데이터 공유를 통해 간단히 통신을 할 수 있는데,
# threading 모듈의 처리를 통해서 한번에 한 스레드만 메모리 영역에 접근할 수 있습니다.
# 각 프로세스가 독립적인 스택, 힙, 코드, 데이터 영역을 가지게 되고
# 한 프로세스에 속한 스레드는 스택 영역을 제외한 메모리 영역을 공유합니다.

# 파이썬 내부에는 스레드 매케니즘이 있지만 병렬 실행을 지원하지 않습니다.
# 스레느는 병렬로 사용하는 것이 가능하고 요즘 사용하는 운영체제에서
# 효율적으로 사용합니다.

In [6]:
# subprocessing 모듈
# 부모 - 자식 프로세서 쌍을 생성
# 부모 프레서스는 사용자를 통해 실행되고 차례대로 다른 일을 처리하는 자식 프로세스의
# 인스턴스까지 실행시킵니다.
# 자식 프로세스를 실행하면서 멀티 코어의 이점을 최대한 가져오고
# 동시성 문제를 운영체제가 알아서 처리하도록 합니다.
import subprocess
f = open('text1.txt', 'w')
# 텍스트 문자의 데이터를 읽고 쓰는데 보통 사용을 합니다.

In [7]:
#threading모듈
# 스레드가 여러개있다면 스데르간의 데이터 공유의 복잡성을 증가하게 합니다.
# 락, 데드락을 회피하는 노력을 해야한다.
# 파이썬은 단 하나의 메인 스레드가 존재합니다.
# 만일 멀티 스레드를 사용하려면 threading 모듈을 사용해야 합니다.

# 내부적으로 락을 관리하기위해서는
# queue 모듈을 사용하며 이는 직렬화가 가능해집니다.

import queue
import threading
q = queue.Queue()

# 워커 스레드
# 작업이 완료되었는데 프로그램이 종료되지 않았다?
# 데몬(Daemon)으로 변환해서 덴몬 스레드가 실행되지 않으면 프로그램을 바로 종료

def worker(num):
    while True:
        # item이라는 변수 안에 큐가 가진 값을 넣습니다.
        item = q.get()
        # 만일 item에 아무런 값이 없다면
        if item is None:
            # 반복문 탈출
            break
        # 스레드의 번호와 현재 가지고 있는 값을 출력합니다.
        print("스레드 {0} : 처리완료 {1}".format(num+1, item))
    # 이전에 큐에 넣은 작업이 확인되었음을 큐에 알립니다.
    q.task_done()
    
if __name__ == "__main__":
    # 워커 스레드를 5만큼 줍니다.
    worker_thread = 5
    # 스레드를 리스트형태로 만듭니다.
    threads = []
    for i in range(worker_thread):
        # 스레드를 생성합니다.
        t = threading.Thread(target=worker, args=(i,))
        # 생성한 스레드를 사용합니다.
        t.start()
        # 스레드 리스트에 생성한 스레드를 추가합니다.
        threads.append(t)
    for item in range(20):
        # put을 이용해서 값을 넣습니다.
        q.put(item)
        # 큐가 가진 모든 정보를 꺼내놓고 대기합니다.
    q.join()
    # 워커스레드를 종료
    for i in range(worker_thread):
        # 워커스레드의 개수만큼 None을 넣습니다
        q.put(None)
    for t in threads:
        # 스레드가 가지고 있는 값들이 완료될때까지 접근할수 없도록 블럭하고
        # 대기하다가 블럭을 없앱니다,
        t.join()

In [10]:
# 뮤텍스
# 락과 같습니다.
# 공유리소스에 한번에 하나의 스레드만 접근할수있도록 동시성 제어 정책을
# 강제하기 위해서 설계되었습니다.
# 1부터 시작하는 정수이며, 배열을 변경할 때마다 '뮤텍스를 잠급니다.'

from threading import Thread, Lock
import threading

def worker(mutax, data, thread_safe):
    if thread_safe:
        # 락을 생성
        mutax.acquire()
    try:
        # threading.get_ident() : 현재 가르키고 있는 스레드의 식별자 정보를 반환
        # data : 들어오는 값을 반환
        print("스레드 {0} : {1} \n".format(threading.get_ident(), data))
    finally:
        if thread_safe:
            # 락을 풀어줍니다.
            mutax.release()
            
if __name__ == "__main__":
    threads  = []
    thread_safe = True
    mutax = Lock()
    for i in range(20):
        # args : 튜플
        # kwargs : 딕셔너리
        t = Thread(target=worker, args=(mutax, i, thread_safe))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()

스레드 16272 : 0 

스레드 22228 : 1 

스레드 19248 : 2 

스레드 18912 : 3 

스레드 7672 : 4 

스레드 12184 : 5 

스레드 3896 : 6 

스레드 21940 : 7 

스레드 20872 : 8 

스레드 16556 : 9 

스레드 4012 : 10 

스레드 1928 : 11 

스레드 7880 : 12 

스레드 21792 : 13 

스레드 21348 : 14 

스레드 6880 : 15 

스레드 292 : 16 

스레드 2680 : 17 

스레드 7172 : 18 

스레드 19620 : 19 



In [2]:
# 세마포어
# 1보다 더 큰수로 시작합니다.
# 세마포어값은 한번에 element를 접근할 수 있는 스레드 수
# 뮤텍스가 가지고 있는 락, 언락 작업과 유사한
# 대기, 신호 작업을 지원하고 있다.

import threading
import time

class threadPool(object):
    def __init__(self):
        self.active = []
        # 락 변수를 thread가 가지고 있는 lock 메서드를 넣습니다.
        self.lock = threading.Lock()
        
    def acquire(self, name):
        with self.lock:
            #  스레드의 이름을 추가합니다.
            self.active.append(name)
            print("획득 : {0} | 스레드풀 : {1}".format(name, self.active))
    def release(self, name):
        with self.lock:
            # 스레드의 이름을 해제합니다.
            self.active.remove(name)
            print("반환 : {0} | 스레드풀 : {1}".format(name, self.active))

def worker(semaphore, pool):
    with semaphore:
        # name에 현재 스레드가 가르키고 있는 스레드의 이름을 name에 넣습니다.
        name = threading.currentThread().getName()
        # 이름을 리스트에 삽입합니다.
        pool.acquire(name)
        # time.sleep() : 대기하는 메서드입니다.
        time.sleep(1)
        # 리스트에 있는 이름을 해제합니다.
        pool.release(name)
        
if __name__ == "__main__":
    threads = []
    pool = threadPool()
    # 세마포어를 10개 넣는다.
    # 값을 10만큼 넣어준다.
    semaphore = threading.Semaphore(10)
    for i in range(10):
        t = threading.Thread(target=worker, args=(semaphore, pool),
                            name = '스레드 ' + str(i))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    
            
            

획득 : 스레드 0 | 스레드풀 : ['스레드 0']
획득 : 스레드 1 | 스레드풀 : ['스레드 0', '스레드 1']
획득 : 스레드 2 | 스레드풀 : ['스레드 0', '스레드 1', '스레드 2']
획득 : 스레드 3 | 스레드풀 : ['스레드 0', '스레드 1', '스레드 2', '스레드 3']
획득 : 스레드 4 | 스레드풀 : ['스레드 0', '스레드 1', '스레드 2', '스레드 3', '스레드 4']
획득 : 스레드 5 | 스레드풀 : ['스레드 0', '스레드 1', '스레드 2', '스레드 3', '스레드 4', '스레드 5']
획득 : 스레드 6 | 스레드풀 : ['스레드 0', '스레드 1', '스레드 2', '스레드 3', '스레드 4', '스레드 5', '스레드 6']
획득 : 스레드 7 | 스레드풀 : ['스레드 0', '스레드 1', '스레드 2', '스레드 3', '스레드 4', '스레드 5', '스레드 6', '스레드 7']
획득 : 스레드 8 | 스레드풀 : ['스레드 0', '스레드 1', '스레드 2', '스레드 3', '스레드 4', '스레드 5', '스레드 6', '스레드 7', '스레드 8']
획득 : 스레드 9 | 스레드풀 : ['스레드 0', '스레드 1', '스레드 2', '스레드 3', '스레드 4', '스레드 5', '스레드 6', '스레드 7', '스레드 8', '스레드 9']
반환 : 스레드 0 | 스레드풀 : ['스레드 1', '스레드 2', '스레드 3', '스레드 4', '스레드 5', '스레드 6', '스레드 7', '스레드 8', '스레드 9']
반환 : 스레드 1 | 스레드풀 : ['스레드 2', '스레드 3', '스레드 4', '스레드 5', '스레드 6', '스레드 7', '스레드 8', '스레드 9']
반환 : 스레드 2 | 스레드풀 : ['스레드 3', '스레드 4', '스레드 5', '스레드 6', '스레드 7', '스레드 8', '스레드 9']
반환 : 스레드 3 | 스레드

In [ ]:
# 데드락
# 두개 이상의 프로세스나 스레드가 서로 작업이 끝나기를 기다리고 있어서
# 결과적으로 아무것도 완료되지 않은 상태를 뜻합니다.
# 데드락을 해결하는 4가지 방법
# 1) 상호 배제 : element가 한번에 한 프로세스만 사용할 수 있다.
# 2) 점유와 대기 ; 프로세스가 element를 가지고 잇는 상태에서 다른 프로세스를 사용하고
# 있는 element의 반납을 기다리는 상태
# 비선점 ; 다른 프로세서가 이미 가지고 있는 element를 강제로 뺏어오지 못하게 하는것
# 순환 대기 : 프로세스 A, B, C가 있다고 했을때, A는 B가 가지고 있는 element를
# B는 C가 가지고 있는 element, C는 A가 가지고 있는 element를 대기하는 상태